In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

In [2]:
torch.cuda.is_available()


True

In [3]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)


tensor([[0.4939, 0.7339, 0.0583],
        [0.8826, 0.1601, 0.6537],
        [0.5201, 0.1196, 0.3131],
        [0.5689, 0.4927, 0.0398],
        [0.8533, 0.1862, 0.9848]])


In [4]:
torch.cuda.is_available()


True

In [5]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [6]:
torch.cuda.is_available()


True

In [7]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join(r"C:\Users\kanna\Desktop\study\Demi", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [9]:
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [10]:
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

In [11]:
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [12]:
MAX_LENGTH = 10  # Maximum sentence length to consider

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [13]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


In [14]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  33,   67,   25, 1387,  908],
        [ 239,  120,  356,    4,    4],
        [ 318, 4022,    7,    2,    2],
        [ 122,    4,    4,    0,    0],
        [  34,    2,    2,    0,    0],
        [   4,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([7, 5, 5, 3, 3])
target_variable: tensor([[  40,   27,   50,  271,   70],
        [  83,  132,    6,  117, 3951],
        [ 101,  948,    2,   76, 2009],
        [ 176,   76,    0,    6,  572],
        [ 905,    4,    0,    2,    6],
        [1984,  948,    0,    0,    2],
        [   4,  619,    0,    0,    0],
        [   2,    4,    0,    0,    0],
        [   0,    2,    0,    0,    0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [ True,  True, False,  True,  True],
        [ True,  True, False, False,  True],
        [ True, 

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu())
        #packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

In [16]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)
    

In [17]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.gru(embedded, last_hidden)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

In [18]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [19]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    loss.backward()

    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [20]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    '''if loadFilename:
        start_iteration = checkpoint['iteration'] + 1'''

    # Training loop
    print("Training...")
    print(start_iteration)
    print(n_iteration)
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))


In [21]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        return all_tokens, all_scores

In [22]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    indexes_batch = [indexesFromSentence(voc, sentence)]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    tokens, scores = searcher(input_batch, lengths, max_length)
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words
from firebase import firebase
firebase = firebase.FirebaseApplication('https://demi-19b11.firebaseio.com/', None)
def evaluateInput(encoder, decoder, searcher, voc):   
    result = firebase.get('/input/', '')
    x=len(result)
    input_sentence = ''
    while(1):
        try:
            while(x==len(result)):
                result = firebase.get('/input/', '')

            j=0
            x=len(result)
            for i in result:
                if j==x-1:
                   input_sentence = result[i]
                j+=1
            #input_sentence = input('> ')
            if input_sentence == 'q' or input_sentence == 'quit':
                break 
            elif 'your name' in input_sentence:
                firebase.post('/output/',"I'm Demi and I'll be there for you")
            else:
                input_sentence = normalizeString(input_sentence)
                output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
                output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
                firebase.post('/output/'," ".join(output_words))


        except KeyError:
            firebase.post('/output/',"I don't know but whatever dude . . .")

In [23]:
from firebase import firebase
firebase = firebase.FirebaseApplication('https://demi-19b11.firebaseio.com/', None)
data="bih"
result = firebase.post('/output/', 'hello')


In [24]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

loadFilename = None
checkpoint_iter = 4000
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                            '{}_checkpoint.tar'.format(checkpoint_iter))


if loadFilename:
    checkpoint = torch.load(loadFilename)
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


#### clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

encoder.train()
decoder.train()


print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()


print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

In [32]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
1
4000
Iteration: 1; Percent complete: 0.0%; Average loss: 1.0975
Iteration: 2; Percent complete: 0.1%; Average loss: 0.8989
Iteration: 3; Percent complete: 0.1%; Average loss: 1.0793
Iteration: 4; Percent complete: 0.1%; Average loss: 0.9310
Iteration: 5; Percent complete: 0.1%; Average loss: 1.0691
Iteration: 6; Percent complete: 0.1%; Average loss: 1.0177
Iteration: 7; Percent complete: 0.2%; Average loss: 0.9196
Iteration: 8; Percent complete: 0.2%; Average loss: 1.0386
Iteration: 9; Percent complete: 0.2%; Average loss: 0.9424
Iteration: 10; Percent complete: 0.2%; Average loss: 1.1218
Iteration: 11; Percent complete: 0.3%; Average loss: 0.9523
Iteration: 12; Percent complete: 0.3%; Average loss: 0.9790
Iteration: 13; Percent complete: 0.3%; Average loss: 1.0044
Iteration: 14; Percent complete: 0.4%; Average loss: 1.0484
Iteration: 15; Percent complete: 0.4%; Average loss: 1.0383
Iteration: 16; Percent complet

Iteration: 136; Percent complete: 3.4%; Average loss: 0.8691
Iteration: 137; Percent complete: 3.4%; Average loss: 1.1126
Iteration: 138; Percent complete: 3.5%; Average loss: 0.9337
Iteration: 139; Percent complete: 3.5%; Average loss: 0.9866
Iteration: 140; Percent complete: 3.5%; Average loss: 1.0083
Iteration: 141; Percent complete: 3.5%; Average loss: 0.8407
Iteration: 142; Percent complete: 3.5%; Average loss: 0.9413
Iteration: 143; Percent complete: 3.6%; Average loss: 0.9479
Iteration: 144; Percent complete: 3.6%; Average loss: 1.0248
Iteration: 145; Percent complete: 3.6%; Average loss: 0.9761
Iteration: 146; Percent complete: 3.6%; Average loss: 1.0752
Iteration: 147; Percent complete: 3.7%; Average loss: 1.1679
Iteration: 148; Percent complete: 3.7%; Average loss: 0.8679
Iteration: 149; Percent complete: 3.7%; Average loss: 0.9686
Iteration: 150; Percent complete: 3.8%; Average loss: 0.8890
Iteration: 151; Percent complete: 3.8%; Average loss: 1.0056
Iteration: 152; Percent 

Iteration: 271; Percent complete: 6.8%; Average loss: 1.1377
Iteration: 272; Percent complete: 6.8%; Average loss: 1.0538
Iteration: 273; Percent complete: 6.8%; Average loss: 1.1230
Iteration: 274; Percent complete: 6.9%; Average loss: 0.9717
Iteration: 275; Percent complete: 6.9%; Average loss: 1.1082
Iteration: 276; Percent complete: 6.9%; Average loss: 0.9627
Iteration: 277; Percent complete: 6.9%; Average loss: 0.8931
Iteration: 278; Percent complete: 7.0%; Average loss: 1.1323
Iteration: 279; Percent complete: 7.0%; Average loss: 1.1865
Iteration: 280; Percent complete: 7.0%; Average loss: 0.8998
Iteration: 281; Percent complete: 7.0%; Average loss: 0.7888
Iteration: 282; Percent complete: 7.0%; Average loss: 1.0513
Iteration: 283; Percent complete: 7.1%; Average loss: 0.8265
Iteration: 284; Percent complete: 7.1%; Average loss: 1.0774
Iteration: 285; Percent complete: 7.1%; Average loss: 1.0251
Iteration: 286; Percent complete: 7.1%; Average loss: 0.8695
Iteration: 287; Percent 

Iteration: 406; Percent complete: 10.2%; Average loss: 1.0095
Iteration: 407; Percent complete: 10.2%; Average loss: 0.9392
Iteration: 408; Percent complete: 10.2%; Average loss: 0.9010
Iteration: 409; Percent complete: 10.2%; Average loss: 1.0003
Iteration: 410; Percent complete: 10.2%; Average loss: 1.0917
Iteration: 411; Percent complete: 10.3%; Average loss: 0.9657
Iteration: 412; Percent complete: 10.3%; Average loss: 0.8754
Iteration: 413; Percent complete: 10.3%; Average loss: 0.8797
Iteration: 414; Percent complete: 10.3%; Average loss: 0.8951
Iteration: 415; Percent complete: 10.4%; Average loss: 1.0037
Iteration: 416; Percent complete: 10.4%; Average loss: 0.8662
Iteration: 417; Percent complete: 10.4%; Average loss: 0.8746
Iteration: 418; Percent complete: 10.4%; Average loss: 0.9664
Iteration: 419; Percent complete: 10.5%; Average loss: 0.8672
Iteration: 420; Percent complete: 10.5%; Average loss: 0.8531
Iteration: 421; Percent complete: 10.5%; Average loss: 0.9913
Iteratio

Iteration: 539; Percent complete: 13.5%; Average loss: 0.9170
Iteration: 540; Percent complete: 13.5%; Average loss: 1.1257
Iteration: 541; Percent complete: 13.5%; Average loss: 0.8750
Iteration: 542; Percent complete: 13.6%; Average loss: 0.8024
Iteration: 543; Percent complete: 13.6%; Average loss: 0.9696
Iteration: 544; Percent complete: 13.6%; Average loss: 0.8915
Iteration: 545; Percent complete: 13.6%; Average loss: 0.8847
Iteration: 546; Percent complete: 13.7%; Average loss: 0.8402
Iteration: 547; Percent complete: 13.7%; Average loss: 0.8869
Iteration: 548; Percent complete: 13.7%; Average loss: 0.8065
Iteration: 549; Percent complete: 13.7%; Average loss: 1.0362
Iteration: 550; Percent complete: 13.8%; Average loss: 1.0788
Iteration: 551; Percent complete: 13.8%; Average loss: 0.9571
Iteration: 552; Percent complete: 13.8%; Average loss: 0.8806
Iteration: 553; Percent complete: 13.8%; Average loss: 0.9082
Iteration: 554; Percent complete: 13.9%; Average loss: 0.9857
Iteratio

...

Iteration: 3428; Percent complete: 85.7%; Average loss: 0.6097
Iteration: 3429; Percent complete: 85.7%; Average loss: 0.5119
Iteration: 3430; Percent complete: 85.8%; Average loss: 0.6952
Iteration: 3431; Percent complete: 85.8%; Average loss: 0.6910
Iteration: 3432; Percent complete: 85.8%; Average loss: 0.7187
Iteration: 3433; Percent complete: 85.8%; Average loss: 0.6285
Iteration: 3434; Percent complete: 85.9%; Average loss: 0.5447
Iteration: 3435; Percent complete: 85.9%; Average loss: 0.5453
Iteration: 3436; Percent complete: 85.9%; Average loss: 0.6590
Iteration: 3437; Percent complete: 85.9%; Average loss: 0.5571
Iteration: 3438; Percent complete: 86.0%; Average loss: 0.5960
Iteration: 3439; Percent complete: 86.0%; Average loss: 0.5799
Iteration: 3440; Percent complete: 86.0%; Average loss: 0.5752
Iteration: 3441; Percent complete: 86.0%; Average loss: 0.7481
Iteration: 3442; Percent complete: 86.1%; Average loss: 0.6855
Iteration: 3443; Percent complete: 86.1%; Average loss:

Iteration: 3559; Percent complete: 89.0%; Average loss: 0.6447
Iteration: 3560; Percent complete: 89.0%; Average loss: 0.6665
Iteration: 3561; Percent complete: 89.0%; Average loss: 0.5605
Iteration: 3562; Percent complete: 89.0%; Average loss: 0.5026
Iteration: 3563; Percent complete: 89.1%; Average loss: 0.7741
Iteration: 3564; Percent complete: 89.1%; Average loss: 0.6277
Iteration: 3565; Percent complete: 89.1%; Average loss: 0.5565
Iteration: 3566; Percent complete: 89.1%; Average loss: 0.7556
Iteration: 3567; Percent complete: 89.2%; Average loss: 0.7360
Iteration: 3568; Percent complete: 89.2%; Average loss: 0.7440
Iteration: 3569; Percent complete: 89.2%; Average loss: 0.6892
Iteration: 3570; Percent complete: 89.2%; Average loss: 0.5004
Iteration: 3571; Percent complete: 89.3%; Average loss: 0.6832
Iteration: 3572; Percent complete: 89.3%; Average loss: 0.7458
Iteration: 3573; Percent complete: 89.3%; Average loss: 0.6335
Iteration: 3574; Percent complete: 89.3%; Average loss:

Iteration: 3690; Percent complete: 92.2%; Average loss: 0.6163
Iteration: 3691; Percent complete: 92.3%; Average loss: 0.6464
Iteration: 3692; Percent complete: 92.3%; Average loss: 0.6554
Iteration: 3693; Percent complete: 92.3%; Average loss: 0.5530
Iteration: 3694; Percent complete: 92.3%; Average loss: 0.6443
Iteration: 3695; Percent complete: 92.4%; Average loss: 0.5654
Iteration: 3696; Percent complete: 92.4%; Average loss: 0.5508
Iteration: 3697; Percent complete: 92.4%; Average loss: 0.6262
Iteration: 3698; Percent complete: 92.5%; Average loss: 0.5995
Iteration: 3699; Percent complete: 92.5%; Average loss: 0.5092
Iteration: 3700; Percent complete: 92.5%; Average loss: 0.7437
Iteration: 3701; Percent complete: 92.5%; Average loss: 0.6798
Iteration: 3702; Percent complete: 92.5%; Average loss: 0.7155
Iteration: 3703; Percent complete: 92.6%; Average loss: 0.6427
Iteration: 3704; Percent complete: 92.6%; Average loss: 0.5168
Iteration: 3705; Percent complete: 92.6%; Average loss:

Iteration: 3821; Percent complete: 95.5%; Average loss: 0.7549
Iteration: 3822; Percent complete: 95.5%; Average loss: 0.6839
Iteration: 3823; Percent complete: 95.6%; Average loss: 0.6139
Iteration: 3824; Percent complete: 95.6%; Average loss: 0.6568
Iteration: 3825; Percent complete: 95.6%; Average loss: 0.6450
Iteration: 3826; Percent complete: 95.7%; Average loss: 0.6207
Iteration: 3827; Percent complete: 95.7%; Average loss: 0.6485
Iteration: 3828; Percent complete: 95.7%; Average loss: 0.6180
Iteration: 3829; Percent complete: 95.7%; Average loss: 0.5988
Iteration: 3830; Percent complete: 95.8%; Average loss: 0.7508
Iteration: 3831; Percent complete: 95.8%; Average loss: 0.5798
Iteration: 3832; Percent complete: 95.8%; Average loss: 0.8059
Iteration: 3833; Percent complete: 95.8%; Average loss: 0.5454
Iteration: 3834; Percent complete: 95.9%; Average loss: 0.5764
Iteration: 3835; Percent complete: 95.9%; Average loss: 0.6618
Iteration: 3836; Percent complete: 95.9%; Average loss:

Iteration: 3952; Percent complete: 98.8%; Average loss: 0.6396
Iteration: 3953; Percent complete: 98.8%; Average loss: 0.5176
Iteration: 3954; Percent complete: 98.9%; Average loss: 0.7342
Iteration: 3955; Percent complete: 98.9%; Average loss: 0.5937
Iteration: 3956; Percent complete: 98.9%; Average loss: 0.6281
Iteration: 3957; Percent complete: 98.9%; Average loss: 0.6823
Iteration: 3958; Percent complete: 99.0%; Average loss: 0.7038
Iteration: 3959; Percent complete: 99.0%; Average loss: 0.6954
Iteration: 3960; Percent complete: 99.0%; Average loss: 0.6501
Iteration: 3961; Percent complete: 99.0%; Average loss: 0.5922
Iteration: 3962; Percent complete: 99.1%; Average loss: 0.6511
Iteration: 3963; Percent complete: 99.1%; Average loss: 0.6023
Iteration: 3964; Percent complete: 99.1%; Average loss: 0.4589
Iteration: 3965; Percent complete: 99.1%; Average loss: 0.6241
Iteration: 3966; Percent complete: 99.2%; Average loss: 0.7469
Iteration: 3967; Percent complete: 99.2%; Average loss:

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\kanna\\Desktop'

In [ ]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

evaluateInput(encoder, decoder, searcher, voc)


In [ ]:
pip install dill


In [ ]:
import dill
dill.dump_session('notebook_env.db')

In [ ]:
import dill
dill.load_session('notebook_env.db')